In [130]:
#!pip install requests_toolbelt

In [12]:
import json
import requests
from lxml import html
from time import sleep
import random
from requests_toolbelt.multipart.encoder import MultipartEncoder
import pandas as pd
import os

In [13]:
url_RUCT = "https://www.educacion.gob.es/ruct/consultaestudios.action"

In [25]:
response = requests.get(url_RUCT)
root = html.fromstring(response.text)
select_universidades = root.xpath('//select[@id="codigoUniversidad"]')
select_codigoTipo = root.xpath('//select[@id="codigoTipo"]')
options_select_codigoTipo = root.xpath('//select[@id="codigoTipo"]/option')
options_select_universidades = root.xpath('//select[@id="codigoUniversidad"]/option')

list_universidades_ = select_universidades[0].text_content().strip().splitlines()
list_codigoTipo_ = select_codigoTipo[0].text_content().strip().splitlines()

list_universidades_.remove("Todos")
list_codigoTipo_.remove("Todos")

list_universidades = []
for univ in list_universidades_:
    list_universidades.append(univ.strip())
    
list_codigoTipo = []
for codigo in list_codigoTipo_:
    list_codigoTipo.append(codigo.strip())

options_select_codigoTipo.pop(0)
list_options_select_codigoTipo = []
for option in options_select_codigoTipo:
    list_options_select_codigoTipo.append(option.attrib["value"])

options_select_universidades.pop(0)
list_options_select_universidades = []
for option in options_select_universidades:
    list_options_select_universidades.append(option.attrib["value"])

#res = {test_keys[i]: test_values[i] for i in range(len(test_keys))}
dict_universidades = {list_options_select_universidades[i]: list_universidades[i] for i in range(len(list_options_select_universidades))}
dict_codigoTipoTitulacion = {list_options_select_codigoTipo[i]: list_codigoTipo[i] for i in range(len(list_options_select_codigoTipo))}

In [26]:
#dict_universidades = {"061": "MU","020":"EHU"}
#dict_codigoTipoTitulacion = {"M" : "Másters"}


In [29]:
columnas_df_RUCT = ["Código", "Título", "Universidad", "Nivel académico", "Estado", "URL"]
df_RUCT = pd.DataFrame(columns = columnas_df_RUCT)

for codigo_universidad, nombre_universidad in dict_universidades.items():
    print("Procesando... " + nombre_universidad)
    for codigoTipoTitulacion, TipoTitulacion in dict_codigoTipoTitulacion.items():
        print("   Procesando " + TipoTitulacion)
        
        url2 = "https://www.educacion.gob.es/ruct/listaestudios?codigoEstado=&d-1335801-e=2&consulta=1&descripcionEstudio=&codigoTipo=" + codigoTipoTitulacion + "&codigoEstudio=&codigoRama=&situacion=&6578706f7274=1&buscarHistorico=N&action:listaestudios=Consultar&actual=estudios&codigoSubTipo=&codigoUniversidad=" + codigo_universidad
        #print(url2)
        params = {
            "codigoUniversidad": codigo_universidad,
            "codigoTipo": codigoTipoTitulacion,
            'action:listaestudios' : 'clicked',
        }

        data = MultipartEncoder(fields = params)

        headers = {
            'Content-type': data.content_type
        }
        response = requests.post(
            'https://www.educacion.gob.es/ruct/consultaestudios.action?actual=estudios',
            data = data,
            headers = headers
        )
        root2 = html.fromstring(response.text)
        tabla_titulos_ = []
        tabla_titulos_ = root2.xpath('//table[@id="estudio"]')
        if len(tabla_titulos_) > 0 :
            #print("Hay tabla")
            filename = 'ruct.xlsx'
            response = requests.get(url2)
            open(filename, 'wb').write(response.content)
            sleep(2)
            df_table_RUCT = pd.read_excel(filename)
            #display(df_table_RUCT)
            df_table_RUCT["URL"] = "https://www.educacion.gob.es/ruct/estudio.action?codigoCiclo=SC&codigoTipo=G&CodigoEstudio=" + df_table_RUCT["Código"].astype(str) + "&actual=estudios"

            df_RUCT = pd.concat([df_RUCT, df_table_RUCT], ignore_index=True)
            print("      Procesado correctamente.")
            os.remove("ruct.xlsx")
        else:
            print("      No hay tabla")
        sleep(random.randint(1,3))
        

Procesando... Centros autorizados para impartir Enseñanzas Artísticas Superiores
   Procesando Ciclo
      No hay tabla
   Procesando Doctor
      No hay tabla
   Procesando Grado
      No hay tabla
   Procesando Acreditación Institucional
      No hay tabla
   Procesando Máster
      No hay tabla
   Procesando Título Equivalente
      Procesado correctamente.
   Procesando Título Extranjero
      No hay tabla
Procesando... Centros dependientes del Ministerio de Defensa
   Procesando Ciclo
      No hay tabla
   Procesando Doctor
      No hay tabla
   Procesando Grado
      No hay tabla
   Procesando Acreditación Institucional
      No hay tabla
   Procesando Máster
      Procesado correctamente.
   Procesando Título Equivalente
      No hay tabla
   Procesando Título Extranjero
      No hay tabla
Procesando... Centros extranjeros autorizados por la Comunidad Autónoma de Cantabria
   Procesando Ciclo
      No hay tabla
   Procesando Doctor
      No hay tabla
   Procesando Grado
      No

In [30]:
display(df_RUCT)

,Código,Título,Universidad,Nivel académico,Estado,URL,Detalle
0,9000164,Título de Conservación y Restauración de Biene...,Centros autorizados para impartir Enseñanzas A...,Título Equivalente,Publicado en B.O.E.,https://www.educacion.gob.es/ruct/estudio.acti...,NaN
1,9000166,Título de Diseño,Centros autorizados para impartir Enseñanzas A...,Título Equivalente,Publicado en B.O.E.,https://www.educacion.gob.es/ruct/estudio.acti...,NaN
2,9000162,Título Superior de Arte Dramático,Centros autorizados para impartir Enseñanzas A...,Título Equivalente,Publicado en B.O.E.,https://www.educacion.gob.es/ruct/estudio.acti...,NaN
3,9000163,Título Superior de Cerámica,Centros autorizados para impartir Enseñanzas A...,Título Equivalente,Publicado en B.O.E.,https://www.educacion.gob.es/ruct/estudio.acti...,NaN
4,9000165,Título Superior de Danza,Centros autorizados para impartir Enseñanzas A...,Título Equivalente,Publicado en B.O.E.,https://www.educacion.gob.es/ruct/estudio.acti...,NaN
...,...,...,...,...,...,...,...
21492,4315941,Máster Universitario Erasmus Mundus en Sanidad...,(Conjunto Internacional),Máster - RD 1393/2007 (1),Publicado en B.O.E.,https://www.educacion.gob.es/ruct/estudio.acti...,NaN
21493,3000870,Máster Universitario Erasmus Mundus en Viticul...,(Conjunto Internacional),Máster - RD 56/2005 (2),Autorizado por Comunidad Aut&oacute;noma,https://www.educacion.gob.es/ruct/estudio.acti...,(TITULACIÓN EXTINGUIDA)
21494,4310812,Máster Universitario Erasmus Mundus en Viticul...,(Conjunto Internacional),Máster - RD 1393/2007 (1),Publicado en B.O.E.,https://www.educacion.gob.es/ruct/estudio.acti...,NaN
21495,4310285,Máster Universitario Internacional Erasmus Mun...,(Conjunto Internacional),Máster - RD 1393/2007 (1),Publicado en B.O.E.,https://www.educacion.gob.es/ruct/estudio.acti...,NaN


In [31]:
df_RUCT.to_json(r'ruct.json',orient='records')